In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# lebron james 2016-17 season
leb = pd.read_csv('lebron.csv')
i = pd.read_csv('injury_with_score.csv')
leb.head(20)

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2016-10-25,31-300,CLE,NaN,NYK,W (+29),1,32:23,...,8,11,14,0,1,4,3,19,22.2,+26
1,2,2.0,2016-10-28,31-303,CLE,@,TOR,W (+3),1,38:40,...,6,8,7,0,0,5,2,21,14.1,0
2,3,3.0,2016-10-29,31-304,CLE,NaN,ORL,W (+6),1,39:00,...,5,6,9,1,1,2,2,23,20.4,+1
3,4,4.0,2016-11-01,31-307,CLE,NaN,HOU,W (+8),1,35:51,...,10,13,8,0,0,4,4,19,16.5,+15
4,5,5.0,2016-11-03,31-309,CLE,NaN,BOS,W (+6),1,36:22,...,6,7,12,1,0,2,2,30,28.5,+18
5,6,6.0,2016-11-05,31-311,CLE,@,PHI,W (+1),1,35:52,...,7,8,14,2,1,5,1,25,22.0,+11
6,7,7.0,2016-11-08,31-314,CLE,NaN,ATL,L (-4),1,36:52,...,7,9,5,3,0,1,3,23,22.1,+7
7,8,8.0,2016-11-11,31-317,CLE,@,WAS,W (+11),1,38:06,...,10,10,5,2,2,6,1,27,20.3,+7
8,9,9.0,2016-11-13,31-319,CLE,NaN,CHO,W (+7),1,38:42,...,8,8,8,1,1,3,0,19,13.8,+12
9,10,10.0,2016-11-15,31-321,CLE,NaN,TOR,W (+4),1,38:04,...,9,9,15,1,0,5,2,28,28.3,+1


In [3]:
# creates a 1-0 column for whether he rested that day
def rest(df):
    game = np.array(df['G'].fillna(0))
    rested = game == 0
    df['Rest'] = rested*1
    return df

# converts string to int or float
def str_to_int(s):
    if len(re.findall('\d', s)) != 0:
        if '.' in s:
            return float(s)
        return int(s[:2])
    return 0

#conversion ot plus/minus to positive or negative number
def plus_min(s):
    if len(re.findall('\d', s)) == 0:
        return 0
    if s[0] == '-':
        return int(s[1:]) * -1
    if s[0] == '+':
        return int(s[1:]) 
    return 0

# computes running mean and does str to int conversion
def meaner(df, col):
    col_vals = df[col].apply(str_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

# computes running mean, no str to int conversion
def meaner_int(df, col):
    col_vals = df[col].values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

# games did not play conversion
# what's happening is say lebron doesnt play the second game
# the function below returns [1, 1, ...], which is the number of 
# games he played until that point
def games(df, col):
    game = df[col].fillna(0)
    length = len(game)
    new_game = []
    for i in np.arange(length):
        new_game.append(max(game[:(i + 1)]))
    df[col] = new_game
    return df

In [4]:
# pretty much uses above functions to convert all statlines of a player into something
# we can use
def manipulate(df, inj, player):
    df = df.drop(labels=['Tm', 'GS', 'Unnamed: 5', 'Opp', 'PF', 'Unnamed: 7'], axis=1)
    df = rest(df) #adds rest column
    df['Age'] = df['Age'].apply(str_to_int) #removes days, just keeps age year
    df['+/-'] = df['+/-'].apply(plus_min) #plus/minus cleaning
    
    #running means
    df = games(df, 'G')
    df = meaner(df, 'MP')
    df = meaner(df, 'FG')
    df = meaner(df, 'FGA')
    df = meaner(df, '3P')
    df = meaner(df, '3PA')
    df = meaner(df, 'FT')
    df = meaner(df, 'FTA')
    
    #ran into error using meaner for FT%, this is the same thing 
    df['FG%'] = df['FG']/df['FGA']
    df['3P%'] = df['3P']/df['3PA']
    df['FT%'] = df['FT']/df['FTA']
    
    #running means
    df = meaner(df, 'ORB')
    df = meaner(df, 'DRB')
    df = meaner(df, 'TRB')
    df = meaner(df, 'AST')
    df = meaner(df, 'STL')
    df = meaner(df, 'BLK')
    df = meaner(df, 'TOV')
    df = meaner(df, 'PTS')
    df = meaner(df, 'GmSc')
    
    #running means for integer valued column
    df = meaner_int(df, '+/-')
    
    #injury table with this player
    player_i = inj[inj['Relinquised'] == player]
    player_i = player_i[['Date', 'score']]
    
    #merge injury and stats and get injury sco
    df = df.merge(player_i, how='left')
    df['score'] = df['score'].fillna(0)
    df['score'] = np.cumsum(df['score'].values)
    df = df.drop('Date', axis=1)
    return df

In [5]:
# newleb is the new lebron table once manipulated
newleb = manipulate(leb, i, 'LeBron James')

In [6]:
newleb.head(20)

,Rk,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
0,1,1.0,31,32.000000,9.000000,14.000000,0.642857,0.000000,3.000000,0.000000,...,11.000000,14.000000,0.000000,1.000000,4.000000,19.000000,22.200000,26.000000,0,0.0
1,2,2.0,31,35.000000,8.000000,15.000000,0.533333,1.000000,4.500000,0.222222,...,9.500000,10.500000,0.000000,0.500000,4.500000,20.000000,18.150000,13.000000,0,0.0
2,3,3.0,31,36.333333,7.666667,15.333333,0.500000,1.333333,5.000000,0.266667,...,8.333333,10.000000,0.333333,0.666667,3.666667,21.000000,18.900000,9.000000,0,0.0
3,4,4.0,31,36.000000,7.250000,14.500000,0.500000,1.250000,4.500000,0.277778,...,9.500000,9.500000,0.250000,0.500000,3.750000,20.500000,18.300000,10.500000,0,0.0
4,5,5.0,31,36.000000,8.200000,16.000000,0.512500,1.200000,4.400000,0.272727,...,9.000000,10.000000,0.400000,0.400000,3.400000,22.400000,20.340000,12.000000,0,0.0
5,6,6.0,31,35.833333,8.333333,17.166667,0.485437,1.166667,4.500000,0.259259,...,8.833333,10.666667,0.666667,0.500000,3.666667,22.833333,20.616667,11.833333,0,0.0
6,7,7.0,31,35.857143,8.285714,17.142857,0.483333,1.285714,4.571429,0.281250,...,8.857143,9.857143,1.000000,0.428571,3.285714,22.857143,20.828571,11.142857,0,0.0
7,8,8.0,31,36.125000,8.375000,17.250000,0.485507,1.500000,4.500000,0.333333,...,9.000000,9.250000,1.125000,0.625000,3.625000,23.375000,20.762500,10.625000,0,0.0
8,9,9.0,31,36.333333,8.333333,17.666667,0.471698,1.555556,4.555556,0.341463,...,8.888889,9.111111,1.111111,0.666667,3.555556,22.888889,19.988889,10.777778,0,0.0
9,10,10.0,31,36.500000,8.500000,17.400000,0.488506,1.600000,4.600000,0.347826,...,8.900000,9.700000,1.100000,0.600000,3.700000,23.400000,20.820000,9.800000,0,0.0


In [16]:
# notice the differences in the table, make sure we all agree with what was done so we're all on
# the same page and can use manipulate

# next up: we will need injury scores (however we want to define them, in terms of severity, length of injury,
# whatever else). just need to append a player's injury score column to the stats table and we're good.
# for example, we need lebron's RUNNING SUM of his injury score appended to the table above for the 
# 16-17 season and then lebron's stats from 16-17 (including injury stats) can be used for our model.

# UPDATE 11/29: just did it so it adds the injuries for that season and the score associated with it to the table
# with the stats. then i did a cumsum of the injury score because of our logic that injuries add up. the newleb
# table is now ready to use for lebron. now the five of us need to split the number of players and grind the to 
# create the dataset. should take no more than 3-4 hrs, and then we feed it into a neural net and we done. 

# the only problem is that the five of us may need to manipulate like 100 players each to build the dataset
# its just downloading a bunch of csv files and running manipulate from above and then joining tables.

# if we are doing this for both 16-17 and 17-18 season, then that's 200 players for each of us. i say that 
# would take about 3-4 hrs? unless someone's more efficient than me.

In [17]:
por = pd.read_csv('porz.csv')
por.head()

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2016-10-25,21-084,NYK,@,CLE,L (-29),1,32:48,...,3,7,0,1,2,2,5,16,9.8,-21
1,2,2.0,2016-10-29,21-088,NYK,NaN,MEM,W (+7),1,36:22,...,5,5,2,0,1,0,5,21,16.9,+14
2,3,3.0,2016-11-01,21-091,NYK,@,DET,L (-13),1,29:47,...,4,6,1,0,1,0,2,18,11.4,-14
3,4,4.0,2016-11-02,21-092,NYK,NaN,HOU,L (-19),1,27:55,...,6,8,0,1,2,1,4,3,2.8,-8
4,5,5.0,2016-11-04,21-094,NYK,@,CHI,W (+13),1,32:59,...,3,5,1,0,0,0,5,27,20.7,+14


In [20]:
manipulate(por, i, 'Kristaps Porzingis')[60:]

,Rk,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
60,61,51.0,21,26.901639,5.540984,12.409836,0.446499,1.557377,4.147541,0.375494,...,5.934426,1.098361,0.606557,1.590164,1.475410,15.180328,10.526230,-0.721311,0,3.0
61,62,52.0,21,26.967742,5.548387,12.403226,0.447334,1.532258,4.112903,0.372549,...,5.967742,1.096774,0.612903,1.564516,1.451613,15.225806,10.583871,-0.451613,0,3.0
62,63,53.0,21,27.174603,5.603175,12.539683,0.446835,1.539683,4.142857,0.371648,...,6.111111,1.111111,0.603175,1.571429,1.460317,15.365079,10.711111,-0.460317,0,3.0
63,64,54.0,21,27.218750,5.609375,12.531250,0.447631,1.515625,4.078125,0.371648,...,6.125000,1.109375,0.593750,1.562500,1.453125,15.343750,10.695313,-0.593750,0,3.0
64,65,55.0,21,27.353846,5.600000,12.569231,0.445532,1.523077,4.092308,0.372180,...,6.153846,1.123077,0.584615,1.584615,1.461538,15.307692,10.647692,-0.676923,0,3.0
65,66,56.0,21,27.484848,5.621212,12.621212,0.445378,1.545455,4.136364,0.373626,...,6.151515,1.166667,0.575758,1.651515,1.439394,15.348485,10.740909,-0.787879,0,3.0
66,67,57.0,21,27.567164,5.641791,12.671642,0.445230,1.552239,4.164179,0.372760,...,6.208955,1.208955,0.567164,1.701493,1.477612,15.402985,10.788060,-0.820896,0,3.0
67,68,58.0,21,27.514706,5.632353,12.735294,0.442263,1.544118,4.191176,0.368421,...,6.161765,1.235294,0.573529,1.720588,1.485294,15.338235,10.711765,-0.838235,0,3.0
68,69,58.0,21,27.115942,5.550725,12.550725,0.442263,1.521739,4.130435,0.368421,...,6.072464,1.217391,0.565217,1.695652,1.463768,15.115942,10.556522,-0.826087,1,3.0
69,70,59.0,21,27.171429,5.600000,12.600000,0.444444,1.500000,4.128571,0.363322,...,6.142857,1.228571,0.585714,1.671429,1.471429,15.157143,10.621429,-0.914286,0,3.0
